In [1]:
!nvidia-smi

Wed Feb  8 21:16:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    25W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
! git clone https://github.com/iolkhovsky/fcos
%cd fcos/
! git pull origin develop
! git checkout develop
! pwd
! ls

fatal: destination path 'fcos' already exists and is not an empty directory.
/content/fcos
From https://github.com/iolkhovsky/fcos
 * branch            develop    -> FETCH_HEAD
Already up to date.
Already on 'develop'
Your branch is up to date with 'origin/develop'.
/content/fcos
common	 dataset  fcos	    requirements_colab.txt  run.py	   tests
configs  docs	  Makefile  requirements.txt	    Sandbox.ipynb  train.py


In [3]:
!make install_colab

pip3 install -r requirements_colab.txt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import os
from google.colab import drive


DRIVE_ROOT = "/content/drive"
drive.mount(DRIVE_ROOT)
WORKSPACE = "colab_training"

logs_root = os.path.join(DRIVE_ROOT, "MyDrive", WORKSPACE, "logs")
if not os.path.isdir(logs_root):
    os.makedirs(logs_root)
print(f"Logs root: {logs_root}")
os.environ["LOGS_ROOT"] = logs_root
checkpoints_root = os.path.join(DRIVE_ROOT, "MyDrive", WORKSPACE, "checkpoints")
if not os.path.isdir(checkpoints_root):
    os.makedirs(checkpoints_root)
print(f"Checkpoints root: {checkpoints_root}")

dataset_root = os.path.join(DRIVE_ROOT, "MyDrive", WORKSPACE, "dataset")
if not os.path.isdir(checkpoints_root):
    os.makedirs(checkpoints_root)
print(f"Dataset root: {dataset_root}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Logs root: /content/drive/MyDrive/colab_training/logs
Checkpoints root: /content/drive/MyDrive/colab_training/checkpoints
Dataset root: /content/drive/MyDrive/colab_training/dataset


In [5]:
from common.utils import read_yaml, write_yaml


training_config = read_yaml("configs/train.yaml")
training_config["logs"]["path"] = logs_root
training_config["checkpoints"]["path"] = checkpoints_root
training_config["dataset"]["root"] = dataset_root
training_config["dataset"]["download"] = False
training_config["epochs"] = 50
training_config["validation_period"]["value"] = 10
train_config_path = os.path.join(logs_root, "training_config.yaml")
print(f"Training config path: {train_config_path}")
os.environ["TRAINING_CONFIG"] = train_config_path
write_yaml(training_config, train_config_path)
training_config

Training config path: /content/drive/MyDrive/colab_training/logs/training_config.yaml


{'model': {'backbone': 'resnet50',
  'labels': 'VocLabelsCodec',
  'resolution': [256, 256]},
 'dataset': {'train': {'batch_size': 16},
  'val': {'batch_size': 36},
  'root': '/content/drive/MyDrive/colab_training/dataset',
  'download': False},
 'optimizer': {'type': 'SGD', 'parameters': {'lr': '1e-2', 'momentum': 0.9}},
 'scheduler': {'type': 'ExponentialLR',
  'parameters': {'gamma': 0.9, 'last_epoch': -1, 'verbose': False}},
 'epochs': 50,
 'autosave_period': {'value': 10, 'type': 'min'},
 'validation_period': {'value': 10, 'type': 'min'},
 'logs': {'path': '/content/drive/MyDrive/colab_training/logs'},
 'checkpoints': {'path': '/content/drive/MyDrive/colab_training/checkpoints'},
 'gradient_clip': 1.0}

In [6]:
!python train.py --config=/content/drive/MyDrive/colab_training/logs/training_config.yaml

{
    "autosave_period": {
        "type": "min",
        "value": 10
    },
    "checkpoints": {
        "path": "/content/drive/MyDrive/colab_training/checkpoints"
    },
    "dataset": {
        "download": false,
        "root": "/content/drive/MyDrive/colab_training/dataset",
        "train": {
            "batch_size": 16
        },
        "val": {
            "batch_size": 36
        }
    },
    "epochs": 50,
    "gradient_clip": 1.0,
    "logs": {
        "path": "/content/drive/MyDrive/colab_training/logs"
    },
    "model": {
        "backbone": "resnet50",
        "labels": "VocLabelsCodec",
        "resolution": [
            256,
            256
        ]
    },
    "optimizer": {
        "parameters": {
            "lr": "1e-2",
            "momentum": 0.9
        },
        "type": "SGD"
    },
    "scheduler": {
        "parameters": {
            "gamma": 0.9,
            "last_epoch": -1,
            "verbose": false
        },
        "type": "ExponentialLR"
    }